## 1. Project: Parkinsons Telemonitoring Phase 3
##### Name: Kalyan kumar Papani
#####             Ayush VimalKumar Patel
#####             Dhruv Harishkumar Patel
#####             Miliben Vineshkumar Patel
#####             Sirivenni Pati

## 2. Import Statements

In [16]:
import random as R
import math
from dataclasses import dataclass
from matplotlib import pyplot as plt
import pandas as pd
from ucimlrepo import fetch_ucirepo
import csv
import numpy as np
import mysql.connector
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import class_weight
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

## 3. Probabilistic Classifier Code

In [17]:
def connect_to_db():
    return mysql.connector.connect(
        user='root',
        password='Dhruv@12121996',
        host='localhost',
        database='ParkinsonsDB'
    )

def fetch_data_from_table(table_name):
    # Function to fetch data from a specific table
    conn = connect_to_db()
    cursor = conn.cursor()

    query = f"SELECT * FROM {table_name}"

    cursor.execute(query)
    columns = [desc[0] for desc in cursor.description]  
    data = cursor.fetchall()

    df = pd.DataFrame(data, columns=columns)

    conn.close()
    return df

def prepare_data():
    # Retrieve data for classification
    training_data = fetch_data_from_table('Training')
    testing_data = fetch_data_from_table('Testing')

    common_columns = list(set(training_data.columns) & set(testing_data.columns))
    common_columns.remove('sex')  # Exclude target column
    training_data['sex'] = training_data['sex'].astype(float).astype(int)
    testing_data['sex'] = testing_data['sex'].astype(float).astype(int)
    X_train, X_test, y_train, y_test = train_test_split(
        training_data[common_columns],  # Features
        training_data['sex'],           # Target variable
        test_size=0.2,                  # Adjust the test size if needed
        stratify=training_data['sex'],  # Ensure stratified split based on 'sex'
        random_state=42                 # Set a random state for reproducibility
    )
    
    return X_train, y_train, X_test, y_test

In [18]:
def probabilistic_classifier(X_train, y_train, X_test, y_test):
    logistic_model = LogisticRegression(max_iter=1000)
    logistic_model.fit(X_train, y_train)
    predicted_probabilities = logistic_model.predict_proba(X_test)
    
    # Use a threshold to convert probabilities to labels
    threshold = 0.5  # Adjust this threshold if needed
    predicted_labels = (predicted_probabilities[:, 1] >= threshold).astype(int)
    
    confusion_mat = confusion_matrix(y_test, predicted_labels)
    accuracy = accuracy_score(y_test, predicted_labels)
    precision = precision_score(y_test, predicted_labels)
    recall = recall_score(y_test, predicted_labels)
    f1 = f1_score(y_test, predicted_labels)
    
    return confusion_mat, accuracy, precision, recall, f1

In [19]:
X_train, y_train, X_test, y_test = prepare_data()
confusion_mat, acc, prec, rec, f1 = probabilistic_classifier(X_train, y_train, X_test, y_test)

In [20]:
X_train.head()

,NHR,DFA,subject_id,Shimmer_APQ5,Shimmer_dB,motor_UPDRS,age,Shimmer_DDA,total_UPDRS,PPE,...,test_time,Jitter_percent,RPDE,Shimmer,Shimmer_APQ3,Shimmer_APQ11,HNR,Jitter_Abs,Jitter_RAP,Jitter_PPQ5
15511,0.433340,0.63669,36,0.13833,1.546,20.125,62,0.27547,26.964,0.502760,...,14.304,0.04430,0.76178,0.18027,0.09182,0.14169,4.739,0.000241,0.02245,0.03156
13729,0.009698,0.53239,15,0.00543,0.109,15.000,65,0.01515,20.491,0.060347,...,110.510,0.00259,0.42563,0.01109,0.00505,0.00760,32.223,0.000013,0.00124,0.00093
952,0.015888,0.70643,35,0.02170,0.361,34.857,71,0.05721,53.572,0.270450,...,174.420,0.00615,0.44720,0.03777,0.01907,0.03237,19.483,0.000041,0.00280,0.00342
8960,0.016354,0.75601,27,0.04559,0.670,10.633,57,0.12069,14.633,0.333690,...,78.473,0.00805,0.56999,0.07532,0.04023,0.05907,20.814,0.000048,0.00386,0.00373
1607,0.086837,0.72643,41,0.03867,0.618,35.109,68,0.09924,43.687,0.539840,...,161.510,0.01659,0.65900,0.06716,0.03308,0.05994,14.596,0.000110,0.00696,0.00928


In [21]:
X_test.head()

,NHR,DFA,subject_id,Shimmer_APQ5,Shimmer_dB,motor_UPDRS,age,Shimmer_DDA,total_UPDRS,PPE,...,test_time,Jitter_percent,RPDE,Shimmer,Shimmer_APQ3,Shimmer_APQ11,HNR,Jitter_Abs,Jitter_RAP,Jitter_PPQ5
8219,0.014211,0.67342,11,0.01028,0.147,19.016,55,0.02456,23.740,0.21843,...,133.500,0.00385,0.41943,0.01682,0.00819,0.01322,25.426,0.000029,0.00153,0.00177
5628,0.012429,0.69819,26,0.02083,0.276,26.720,49,0.05312,34.533,0.19503,...,94.904,0.00378,0.68728,0.03264,0.01771,0.02542,21.382,0.000035,0.00189,0.00220
8591,0.032067,0.76863,24,0.02135,0.297,13.596,60,0.04556,18.192,0.21671,...,83.797,0.00547,0.50503,0.03326,0.01519,0.03745,17.391,0.000048,0.00225,0.00302
2801,0.009875,0.65258,14,0.00541,0.085,11.875,58,0.01300,16.715,0.21505,...,76.381,0.00367,0.33836,0.00897,0.00433,0.00800,25.095,0.000020,0.00179,0.00212
4604,0.005492,0.54587,1,0.01083,0.175,29.435,72,0.02931,36.870,0.17409,...,40.655,0.00335,0.42962,0.01924,0.00977,0.01701,25.086,0.000017,0.00151,0.00159


In [22]:
y_train.head()

15511    1
13729    0
952      0
8960     1
1607     1
Name: sex, dtype: int32

In [23]:
y_test.head()

8219    0
5628    0
8591    0
2801    1
4604    0
Name: sex, dtype: int32

## 4. Probabilistic Classification Results

In [24]:
print("Confusion Matrix for Probabilistic Classifier:\n", confusion_mat)
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1-score:", f1)

Confusion Matrix for Probabilistic Classifier:
 [[2091  328]
 [ 599  507]]
Accuracy: 0.7370212765957447
Precision: 0.6071856287425149
Recall: 0.45840867992766726
F1-score: 0.5224111282843894


## 5. Euclidean Distance Classifier

In [25]:
def euclidean_distance_classifier(X_train, y_train, X_test, y_test):
    knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
    knn.fit(X_train, y_train)
    predictions_euclidean = knn.predict(X_test)
    
    confusion_matrix_knn = confusion_matrix(y_test, predictions_euclidean)
    
    return predictions_euclidean, confusion_matrix_knn
X_train = np.array(X_train)
X_test = np.array(X_test)

## 6. Euclidean Distance Classification Results

In [26]:
euclidean_predictions, confusion_matrix_euclidean = euclidean_distance_classifier(X_train, y_train, X_test, y_test)
print('Classification Report for Euclidean Distance Classifier:')
print(classification_report(y_test, euclidean_predictions))

Classification Report for Euclidean Distance Classifier:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2419
           1       1.00      1.00      1.00      1106

    accuracy                           1.00      3525
   macro avg       1.00      1.00      1.00      3525
weighted avg       1.00      1.00      1.00      3525



In [27]:
print(confusion_matrix_euclidean)

[[2417    2]
 [   5 1101]]


## 7. Cosine Similarity Classifier

In [28]:
def cosine_similarity_classifier(X_train, y_train, X_test, y_test):
    knn = KNeighborsClassifier(n_neighbors=5, metric='cosine')
    knn.fit(X_train, y_train)
    predictions_cosine = knn.predict(X_test)
    
    confusion_matrix_cosine = confusion_matrix(y_test, predictions_cosine)
    
    return predictions_cosine, confusion_matrix_cosine

## 8. Cosine Similarity Classification Results

In [29]:
cosine_predictions, confusion_matrix_cosine = cosine_similarity_classifier(X_train, y_train, X_test, y_test)
print('Classification Report for Cosine Similarity Classifier:')
print(classification_report(y_test, cosine_predictions))

Classification Report for Cosine Similarity Classifier:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      2419
           1       0.98      0.99      0.98      1106

    accuracy                           0.99      3525
   macro avg       0.99      0.99      0.99      3525
weighted avg       0.99      0.99      0.99      3525



In [30]:
print(confusion_matrix_cosine)

[[2393   26]
 [  11 1095]]
